In [1]:
import numpy as np

def new_CR(index, memory, terminal): #big termninal value is seen as no speical terminal value
    mean = memory[:,0][index]
    if mean == terminal:
        return 0
    else:
        return np.clip(np.random.normal(mean, 0.1), 0, 1)



def new_F(index, memory):
    mean = memory[:,1][index]
    x = mean + np.random.standard_cauchy() * 0.1
    if x>1:
        x = 1
    if x<=0:
        x = new_F(index, memory)
    return x


def binomial_crossover(parent1, parent2, p, des):
    child = []
    for i in range(len(parent1)):
        if np.random.uniform(0, 1) < p or i ==  des:
            child.append(parent1[i])
        else:
            child.append(parent2[i])
    return child

def weighted_lehmer_mean(function_evals, S):
    function_evals = np.array(function_evals)
    diff = np.abs(function_evals[:, 0] - function_evals[:, 1])
    sum_diff = np.sum(diff)

    weights = diff/sum_diff

    num = np.sum([ weights[i] * S[i]**2 for i in range(len(S))])
    dem = np.sum([ weights[i] * S[i] for i in range(len(S))])

    return num / dem

In [2]:
def SHADE(func, bounds, nr_variables, max_iters, pop_size, H, p = 0.11, archive_size = None):
    if archive_size is None:
        archive_size = pop_size


    population = np.random.uniform(bounds[0], bounds[1], (pop_size, nr_variables))
    fitness = np.apply_along_axis(func, 1, population)
    memory = np.array([[0.5, 0.5] for i in range(H)]) #CR in first col and F in second
    archive = []
    
    k = 0
    terminal = 10e+10

    objective = []
    sol = []


    for i in range(max_iters):

        SCR = []
        SF = []

        parent_and_kid_pair = []

        des =  np.random.randint(0, high=nr_variables)


        for j in range(pop_size):
            random_index = np.random.randint(0, H)
            CR = new_CR(random_index, memory, terminal)
            F = new_F(random_index, memory)

            best_index = np.argsort(fitness)[:max(2, int(round(p*pop_size)))]
            p_best = np.random.choice(best_index)

            individual = np.array(population[j].copy())

            parents = [population[j]]
            while len(parents)<2:
                random_row = population[np.random.randint(population.shape[0])]
                if np.array([np.array_equal(parents[i], random_row) for i in range(len(parents))]).any():
                    parents.append(random_row)

            if len(archive) == 0:
                union_rows = population
            else:
                concatenated = np.concatenate((archive, population))
                union_rows = np.unique(concatenated, axis=0)        
            while len(parents)<3:
                random_row = union_rows[np.random.randint(union_rows.shape[0])]
                if np.array([np.array_equal(parents[i], random_row) for i in range(len(parents))]).any():
                    parents.append(random_row)


            mutated = individual + F * (np.array(population[p_best]) - individual)
            mutated += F * (parents[0] - parents[1])
            mutated = np.clip(mutated, bounds[0], bounds[1])

            crossovered = binomial_crossover(mutated, individual, CR, des)
            candidate_fitness = func(crossovered)


            if candidate_fitness <= fitness[j]:
                if len(archive) < archive_size:
                    archive.append(individual)
                else:
                    index = np.random.randint(0, archive_size)
                    del archive[index]
                    archive.append(individual)
                
                SCR.append(CR)
                SF.append(F)

                parent_and_kid_pair.append([fitness[j],candidate_fitness])

                population[j] = crossovered

            if i == 0 and j == 0:
                objective.append(fitness[j])
                sol.append(individual)
            if fitness[j]<=objective[-1]:
                objective.append(fitness[j])
                sol.append(individual)            

        if len(SCR) != 0 and len(SF) != 0:
            if memory[:,0][k] == terminal or max(SCR) == 0:
                memory[:,0][k] = terminal
            else:
                memory[:,0][k] = weighted_lehmer_mean(parent_and_kid_pair, SCR)
                
            memory[:,1][k] = weighted_lehmer_mean(parent_and_kid_pair, SF)

            k += 1
            k = k%H
        else:
            memory[:,0][k] = memory[:,0][k]
            memory[:,1][k] = memory[:,1][k]

    return objective, sol